<a href="https://colab.research.google.com/github/raja-khalil/tradutorArtigosAzureAI/blob/main/trad_art_online.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4 python-docx openai langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13


In [5]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from docx import Document
import os

# Configuração da API do Azure Translator
subscription_key = "CIw47VFyWK4ZxrbiZ8qa9EYLHmK6h47hRK3ypQurkulkHBozQh5zJQQJ99AKACYeBjFXJ3w3AAAAACOGhmae"  # Substitua pela chave válida
endpoint = 'https://api.cognitive.microsofttranslator.com'
location = "eastus"  # Ajuste conforme necessário
language_destination = 'pt-br'  # Idioma de destino

def translator_text(text, target_language):
    """Traduz um texto usando a API do Azure Translator."""
    path = '/translate'
    constructed_url = endpoint + path
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(os.urandom(16))
    }
    body = [{'text': text}]
    params = {
        'api-version': "3.0",
        'from': 'en',
        'to': target_language
    }
    response = requests.post(constructed_url, params=params, headers=headers, json=body)
    response.raise_for_status()  # Levanta erro se algo der errado
    return response.json()[0]["translations"][0]["text"]

def fetch_and_translate(url):
    """Baixa conteúdo de uma URL, traduz, salva e exibe o resultado."""
    # Baixa o conteúdo da URL
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Erro ao acessar a URL: {response.status_code}")

    # Processa o HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.title.string if soup.title else "Documento Sem Título"
    paragraphs = soup.find_all('p')

    # Junta o texto dos parágrafos
    full_text = "\n".join([p.get_text() for p in paragraphs])
    if not full_text.strip():
        raise Exception("Nenhum texto encontrado na página.")

    # Traduz o texto
    translated_text = translator_text(full_text, language_destination)

    # Inclui informações adicionais no cabeçalho
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    header_info = f"URL: {url}\nData e Hora: {timestamp}\n\n"

    # Exibe a tradução no terminal
    print("Texto traduzido:")
    print(header_info)
    print(translated_text)
    print("-" * 50)

    # Cria e salva o arquivo traduzido
    doc = Document()
    doc.add_paragraph(header_info)
    doc.add_paragraph(translated_text)

    filename = f"Traducao_{title.replace(' ', '_')}_{language_destination}.docx"
    doc.save(filename)
    print(f"Documento traduzido salvo em: {filename}")
    return filename

# Exemplo de uso
url_to_translate = "https://dev.to/luca1iu/python-effective-techniques-for-managing-dates-in-dataframe-7n6"  # Substitua pela URL do artigo
try:
    translated_file = fetch_and_translate(url_to_translate)
    print(f"Arquivo salvo: {translated_file}")
except Exception as e:
    print(f"Erro: {e}")


Texto traduzido:
URL: https://dev.to/luca1iu/python-effective-techniques-for-managing-dates-in-dataframe-7n6
Data e Hora: 2024-11-10 13:55:02



Postado em 18 mar

Na análise de dados e no processamento de séries temporais, trabalhar com colunas de data é essencial para extrair insights significativos de conjuntos de dados. Entender como extrair componentes de data específicos, calcular diferenças de horário, definir colunas de data como índices e converter formatos de data são habilidades essenciais para analistas e cientistas de dados. Neste artigo, exploraremos cenários e métodos comuns para lidar com datas de tabela em Python usando a biblioteca pandas. 
Às vezes, precisamos apenas extrair ano, mês, dia e outras informações de uma data para analisar e visualizar melhor os dados. Isso pode ser feito usando o atributo dt:
Por exemplo, podemos extrair o ano de uma coluna de data em um dataframe pandas usando o seguinte código:

Isso produzirá um dataframe com uma coluna de ano adicion